In [9]:
pip install pandas numpy datetime tqdm scikit-learn statsmodels pykalman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.5/248.5 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 KB 17.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
# ---------------------- Imports ---------------------- #
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
import json

# ---------------------- DNS Functions ---------------------- #
def DNS_formula(x, f, lambb):
    l1, s1, c1 = f
    return l1 + s1 * ((1 - np.exp(-lambb * x)) / (lambb * x)) + \
           c1 * (((1 - np.exp(-lambb * x)) / (lambb * x)) - np.exp(-lambb * x))

def params_VAR(factors_df):
    from statsmodels.tsa.api import VAR
    model = VAR(factors_df)
    fitted = model.fit(1)
    coeffs = fitted.params.values
    offset = coeffs[0, :]
    trans_mat = coeffs[1:, :].T
    return offset, trans_mat

def forecast_DNS_KF_explosivcor(da, tau, lamb, state_init, offset_init, trans_init, pred):
    from pykalman import KalmanFilter
    t = np.array(tau).T
    dat = da.to_numpy()

    kf = KalmanFilter(
        transition_matrices=trans_init,
        observation_matrices=np.hstack((np.ones((tau.size, 1)), (1 - np.exp(-lamb * t)) / (lamb * t), (1 - np.exp(-lamb * t)) / (lamb * t) - np.exp(-lamb * t))),
        transition_covariance=np.eye(3),
        observation_covariance=np.eye(dat.shape[1]),
        transition_offsets=offset_init,
        observation_offsets=np.zeros(dat.shape[1]),
        initial_state_mean=state_init,
        initial_state_covariance=np.eye(3),
        em_vars=[
            'transition_matrices', 'transition_offsets',
            'transition_covariance', 'observation_covariance',
            'initial_state_mean', 'initial_state_covariance'
        ],
        n_dim_state=3,
        n_dim_obs=dat.shape[1]
    )
    kf = kf.em(dat, n_iter=20)
    smoother_x, _ = kf.smooth(dat)

    A = kf.transition_matrices
    offs = kf.transition_offsets

    eigvals = np.abs(np.linalg.eigvals(A))
    print(f"[DEBUG] Eigenvalues: {eigvals}")
    
    if np.max(eigvals) > 1.01:
        print("explosivity corrected")
        A = np.eye(3)
        offs = np.zeros_like(offs)

    forecast = np.zeros((pred, 3))
    last_value = np.array([smoother_x[-1]])
    for i in range(pred):
        forecast[i, :] = np.array(last_value @ A.T + offs)
        last_value = forecast[i, :]

    return forecast

def DNS_OLS(data, tau_in, lamb_i):
    tau = tau_in.reshape(-1, 1)
    dummy = lamb_i * tau
    col2 = (1 - np.exp(-dummy)) / dummy
    col3 = col2 - np.exp(-dummy)
    X = np.hstack([np.ones_like(tau), col2, col3])
    XtX_inv_Xt = np.linalg.pinv(X.T @ X) @ X.T
    betas = (XtX_inv_Xt @ data.T).T
    return betas

# ---------------------- Parameters ---------------------- #
window_size = 3 * 252  # 3 years assuming 252 trading days/year
forecast_horizon = 1
lamb_i = 0.496
tau = np.array([0.25, 0.5, 1, 3, 5, 10])

# ---------------------- Load Data ---------------------- #
y_df = pd.read_csv("Y_df.csv", index_col=0, parse_dates=True)
date_index = y_df.index
y_all = y_df.values
columns = y_df.columns.tolist()

# ---------------------- Forecasting Loop ---------------------- #
start_date = pd.to_datetime("2022-04-13")
end_date = pd.to_datetime("2025-03-05")
start_idx = date_index.get_loc(start_date)
end_idx = date_index.get_loc(end_date)

preds, trues, test_dates, results = [], [], [], []

for i in tqdm(range(start_idx, end_idx + 1), desc="Rolling DNS Forecast"):
    y_hist = y_all[i - window_size:i]
    y_true = y_all[i]

    betas = DNS_OLS(y_hist, tau, lamb_i)
    offset, A = params_VAR(pd.DataFrame(betas))
    forecasted_betas = forecast_DNS_KF_explosivcor(
        pd.DataFrame(y_hist), tau.reshape(1, -1), lamb_i,
        state_init=betas[-1], offset_init=offset, trans_init=A,
        pred=forecast_horizon
    )

    y_pred = DNS_formula(tau, forecasted_betas[-1], lamb_i)

    eval_date = date_index[i]
    preds.append(y_pred)
    trues.append(y_true)
    test_dates.append(eval_date)

    results.append({
        "eval_date": eval_date.strftime("%Y-%m-%d"),
        "true_yields": y_true.tolist(),
        "forecast_yields": y_pred.tolist(),
        "forecast_betas": forecasted_betas[-1].tolist(),
        "mae": mean_absolute_error(y_true, y_pred)
    })

# ---------------------- Save Outputs ---------------------- #
pd.DataFrame(trues, columns=columns, index=test_dates).to_csv("dns_fixed_window_true.csv")
pd.DataFrame(preds, columns=columns, index=test_dates).to_csv("dns_fixed_window_preds.csv")

with open("dns_fixed_window_metadata.json", "w") as f:
    json.dump(results, f, indent=4)

pd.DataFrame(results).to_csv("dns_fixed_window_results.csv", index=False)


Rolling DNS Forecast:   0%|          | 1/756 [00:05<1:04:03,  5.09s/it]

[DEBUG] Eigenvalues: [0.96670611 0.9988845  1.0036314 ]


Rolling DNS Forecast:   0%|          | 2/756 [00:10<1:04:12,  5.11s/it]

[DEBUG] Eigenvalues: [0.96667985 0.99898878 1.00310743]


Rolling DNS Forecast:   0%|          | 3/756 [00:15<1:04:06,  5.11s/it]

...

[DEBUG] Eigenvalues: [0.96188475 0.99429497 0.99429497]


Rolling DNS Forecast: 100%|█████████▉| 755/756 [1:03:34<00:05,  5.03s/it]

[DEBUG] Eigenvalues: [0.9619873  0.99409706 0.99409706]


Rolling DNS Forecast: 100%|██████████| 756/756 [1:03:39<00:00,  5.05s/it]

[DEBUG] Eigenvalues: [0.96169901 0.9941248  0.9941248 ]
